In [ ]:
import numpy as np
import pandas as pd
from scipy.fftpack import fft, ifft, fftfreq
import scipy.signal as signal
import matplotlib.pyplot as plt
from configparser import ConfigParser

In [ ]:
folder = 'Distortion comparison/'
first = 'UH single trans max amp'
second = 'Single trans max amp'
first_full = '../Measurements/' + folder + first + '/'
second_full = '../Measurements/' + folder + second + '/'

config = ConfigParser()
config.read(first_full + 'settings.ini')
n_1=int(config['General']['blocksize'])
fs_1=float(config['General']['fs'])
f_1=fftfreq(n_1,1/fs_1)

config.read(second_full + 'settings.ini')
n_2=int(config['General']['blocksize'])
fs_2=float(config['General']['fs'])
f_2 = fftfreq(n_2,1/fs_2)

dB = lambda x: 20*np.log10(np.abs(x))
#calibration = 3.55e-3 # mV/Ps
calibration = float(config['General']['Microphone calibration'])
dBuSPL = lambda x: 20*np.log10(np.abs(x)/(20e-6*calibration))
dBu2SPL = lambda x: 10*np.log10(np.abs(x)/(20e-6*calibration)**2)

timedata_1=pd.read_csv(first_full + 'F1.txt',delimiter='\t')
timedata_2=pd.read_csv(second_full + 'F1.txt',delimiter='\t')

spectrogram_1=timedata_1.apply(lambda x: signal.spectrogram(
    x,fs=fs_1,nperseg=2**15,scaling='spectrum'))
spectrogram_2=timedata_2.apply(lambda x: signal.spectrogram(
    x,fs=fs_2,nperseg=2**15,scaling='spectrum'))
win_1 = signal.windows.tukey(n_1,0.25)
win_2 = signal.windows.tukey(n_2,0.25)
win_1 = np.ones(n_1)
win_2 = np.ones(n_2)
frequencydata_1=timedata_1.apply(lambda x: fft(x*win_1),raw=True)/n_1
frequencydata_2=timedata_2.apply(lambda x: fft(x*win_2),raw=True)/n_2


In [ ]:
channel_1='Microphone'
channel_2=channel_1
#channel_2='Amplifier'
plotType = 'Response' # 'Spectrogram' 'Response'
scale = 'SPL' # 'SPL' 'V'
fMin = 0e3
fMax = 500e3
dBMin = None
dBMax = None

if plotType == 'Spectrogram':
    if scale == 'SPL':
        fun = dBu2SPL
    else:
        fun = dB
        
    fIdx = (spectrogram_1[channel_1][0]>fMin) & (spectrogram_1[channel_1][0]<fMax) 
    plt.pcolormesh(spectrogram_1[channel_1][1],
                   spectrogram_1[channel_1][0][fIdx],
                   fun(spectrogram_1[channel_1][2][fIdx]),
                   vmin=dBMin,vmax=dBMax)              
    plt.ylim((fMin,fMax))
    plt.colorbar()
    plt.show()

    fIdx = (spectrogram_2[channel_2][0]>fMin) & (spectrogram_2[channel_2][0]<fMax) 
    plt.pcolormesh(spectrogram_2[channel_2][1],
                   spectrogram_2[channel_2][0][fIdx],
                   fun(spectrogram_2[channel_2][2][fIdx]),
                   vmin=dBMin,vmax=dBMax)            
    plt.ylim((fMin,fMax))
    plt.colorbar()
    plt.show()
else:
    if scale == 'SPL':
        fun = dBuSPL
    else:
        fun= dB
    
    fIdx = (f_1>fMin) & (f_1<fMax)
    plt.plot(f_1[fIdx],fun(frequencydata_1[channel_1][fIdx]))
    plt.ylim(ymin=dBMin,ymax=dBMax)
    plt.show()
    print('Max value in "{}" for "{}": {:.1f} dB ({})'.format(
        channel_1, first, np.max(fun(frequencydata_1[channel_1][fIdx])), scale
        ))

    
    fIdx = (f_2>fMin) & (f_2<fMax)
    plt.plot(f_2[fIdx],fun(frequencydata_2[channel_2][fIdx]))
    plt.ylim(ymin=dBMin,ymax=dBMax)
    plt.show()
    print('Max value in "{}" for "{}": {:.1f} dB ({})'.format(
        channel_2, second, np.max(fun(frequencydata_2[channel_2][fIdx])), scale
        ))
    